In [159]:
import pandas as pd
import requests
import numpy as np
import nltk
from bs4 import BeautifulSoup


In [160]:
df=pd.read_excel('input.xlsx')
df.head(10)

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...
5,6,https://insights.blackcoffer.com/how-are-genet...
6,7,https://insights.blackcoffer.com/how-is-ai-use...
7,8,https://insights.blackcoffer.com/benefits-of-b...
8,9,https://insights.blackcoffer.com/how-big-data-...
9,10,https://insights.blackcoffer.com/how-will-ai-m...


In [161]:
links=list(df.iloc[:,1])
links

['https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/',
 'https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/',
 'https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/',
 'https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-successfully-treated-in-healthcare/',
 'https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/',
 'https://insights.blackcoffer.com/how-are-genetic-sequencing-maps-affected-by-deep-learning-and-ai/',
 'https://insights.blackcoffer.com/how-is-ai-used-to-solve-traffic-management/',
 'https://insights.blackcoffer.com/benefits-of-big-data-in-different-fields/',
 'https://insights.blackcoffer.com/how-big-data-will-impact-the-future-of-business/',
 'https://insights.blackcoffer.com/how-will-ai-make-decisions-in-tomorrows-wars/',
 'https://insights.blackcoffer.com/which-one-is-better-ai-

In [162]:

def get_text(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        article = soup.findAll(attrs={'class': 'td-post-content tagdiv-type'})
        head = soup.findAll('h1', attrs={'class': 'entry-title'})

        text = article[0].text.replace('\n', "") if article else None
        Headline = head[0].text.replace('\n', "") if head else None

        info = pd.DataFrame({'Headline': [Headline], 'Text': [text], 'URL': [url]})
        return info
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [163]:
# Load the Excel file
df = pd.read_excel('input.xlsx')  
# Apply the function to each URL
df_list = [get_text(url) for url in df['URL']]  
# Concatenate all the dataframes
final_df = pd.concat(df_list, ignore_index=True)


In [164]:
final_df.head()

,Headline,Text,URL
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,https://insights.blackcoffer.com/how-is-login-...
1,None,None,https://insights.blackcoffer.com/how-does-ai-h...
2,AI and its impact on the Fashion Industry,If you were a fan of the 90’s film Clueless ba...,https://insights.blackcoffer.com/ai-and-its-im...
3,None,None,https://insights.blackcoffer.com/how-do-deep-l...
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h...",https://insights.blackcoffer.com/how-artificia...


In [165]:
# Load your list of stopwords
with open('StopWords_Geographic.txt', 'r') as f:
    stop_list = set(word.strip() for word in f)

def transform(text):
    # Check if text is not None
    if text is not None:
        # Convert the text to lowercase and split into words
        words = text.lower().split()

        # Remove the stopwords
        words = [word for word in words if word not in stop_list]

        # Join the words back into a string
        text = ' '.join(words)
    
    return text

# Apply the transformation to the 'Text' column and store the result in 'Transform_Text'
final_df['Transform_Text'] = final_df['Text'].apply(transform)




In [166]:
final_df.head()

,Headline,Text,URL,Transform_Text
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,https://insights.blackcoffer.com/how-is-login-...,when people hear ai they often think about sen...
1,None,None,https://insights.blackcoffer.com/how-does-ai-h...,None
2,AI and its impact on the Fashion Industry,If you were a fan of the 90’s film Clueless ba...,https://insights.blackcoffer.com/ai-and-its-im...,if you were a fan of the 90’s film clueless ba...
3,None,None,https://insights.blackcoffer.com/how-do-deep-l...,None
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h...",https://insights.blackcoffer.com/how-artificia...,"from the stone age to the modern world, from h..."


In [167]:
# word count in each text row.
final_df['word_counts'] = final_df['Transform_Text'].apply(lambda x: len(str(x).split()) if x is not None else 0)


In [168]:
final_df['chara_count'] = final_df['Transform_Text'].apply(lambda x: len(str(x))if x is not None else 0)


In [169]:
import nltk
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [170]:
import nltk

# Initialize the new column with NaN values
final_df['average number of words per sentence'] = np.nan

for i in range(len(final_df)):
    text = final_df.at[i, 'Text']
    if text is not None:
        final_df.at[i, 'average number of words per sentence'] = final_df.at[i, 'word_counts'] / len(nltk.sent_tokenize(text))


In [171]:
final_df['average word length'] = np.nan

for i in range(0,len(df)):   
    final_df['average word length'][i] = final_df['chara_count'][i]/final_df['word_counts'][i]

C:\Users\Gaurav\AppData\Local\Temp\ipykernel_9436\3802710702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['average word length'][i] = final_df['chara_count'][i]/final_df['word_counts'][i]
C:\Users\Gaurav\AppData\Local\Temp\ipykernel_9436\3802710702.py:4: RuntimeWarning: invalid value encountered in scalar divide
  final_df['average word length'][i] = final_df['chara_count'][i]/final_df['word_counts'][i]
C:\Users\Gaurav\AppData\Local\Temp\ipykernel_9436\3802710702.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['average word length'][i] = final_df['chara_count'][i]/final_df['word_counts'][i]

In [172]:
final_df.head()

,Headline,Text,URL,Transform_Text,word_counts,chara_count,average number of words per sentence,average word length
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,https://insights.blackcoffer.com/how-is-login-...,when people hear ai they often think about sen...,700,4451,38.888889,6.358571
1,None,None,https://insights.blackcoffer.com/how-does-ai-h...,None,0,0,NaN,NaN
2,AI and its impact on the Fashion Industry,If you were a fan of the 90’s film Clueless ba...,https://insights.blackcoffer.com/ai-and-its-im...,if you were a fan of the 90’s film clueless ba...,1752,11605,44.923077,6.623858
3,None,None,https://insights.blackcoffer.com/how-do-deep-l...,None,0,0,NaN,NaN
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h...",https://insights.blackcoffer.com/how-artificia...,"from the stone age to the modern world, from h...",1208,7545,25.166667,6.245861


In [173]:
#syllable count
def syllable_count(x):
    v = []
    d = {}
    for i in x:
        if i in "aeiou":
            v.append(i)
            d[i] = d.get(i,0)+1
            
    k = []
    for i in d:
        k.append(d[i])

    return np.sum(k)


In [174]:
final_df['syllable per word'] = final_df['Transform_Text'].apply(lambda x: syllable_count(x) if x is not None else 0)

In [175]:
from nltk.corpus import cmudict

# Load the CMU Pronouncing Dictionary
d = cmudict.dict()

def syllable_count(word):
    # Use the CMU Pronouncing Dictionary to get the syllables
    # If the word isn't in the dictionary, assume it has one syllable
    return [len(list(y for y in x if y[-1].isdigit())) for x in d.get(word, [''])][0]

def complex_word_count(text):
    # Split the text into words
    words = text.split()
    
    # Count the complex words
    complex_words = [word for word in words if syllable_count(word) >= 3]
    
    return len(complex_words)

# Apply the function to the 'Transform_Text' column
final_df['complex word count'] = final_df['Transform_Text'].apply(lambda x: complex_word_count(x) if x is not None else 0)


In [176]:

# Initialize the new columns with NaN values
final_df['sentence length'] = np.nan
final_df['Average Sentence Length'] = np.nan
final_df['Percentage of Complex words'] = np.nan
final_df['Fog Index'] = np.nan

for i in range(len(final_df)):
    text = final_df.at[i, 'Text']
    if text is not None:
        final_df.at[i, 'sentence length'] = len(nltk.sent_tokenize(text))
        final_df.at[i, 'Average Sentence Length'] = final_df.at[i, 'word_counts'] / final_df.at[i, 'sentence length']
        final_df.at[i, 'Percentage of Complex words'] = final_df.at[i, 'complex word count'] / final_df.at[i, 'word_counts']
        final_df.at[i, 'Fog Index'] = 0.4 * (final_df.at[i, 'Average Sentence Length'] + final_df.at[i, 'Percentage of Complex words'])

'''final_df. Here’s a breakdown of what it does:
Initialize new columns: The code first initializes four new columns in final_df with np.nan values: ‘sentence length’, ‘Average Sentence Length’, ‘Percentage of Complex words’, and ‘Fog Index’.
Loop over the DataFrame: The code then loops over each row in final_df. For each row, it does the following:
Sentence length: It calculates the number of sentences in the ‘Text’ column using the nltk.sent_tokenize function and stores the result in the ‘sentence length’ column.
Average Sentence Length: It calculates the average sentence length by dividing the total word count (‘word_counts’) by the number of sentences (‘sentence length’) and stores the result in the ‘Average Sentence Length’ column.
Percentage of Complex words: It calculates the percentage of complex words (words with more than two syllables) by dividing the complex word count (‘complex word count’) by the total word count (‘word_counts’) and stores the result in the ‘Percentage of Complex words’ column.
Fog Index: It calculates the Fog Index, a readability metric, using the formula 0.4 * (Average Sentence Length + Percentage of Complex words) and stores the result in the ‘Fog Index '''

'final_df. Here’s a breakdown of what it does:\nInitialize new columns: The code first initializes four new columns in final_df with np.nan values: ‘sentence length’, ‘Average Sentence Length’, ‘Percentage of Complex words’, and ‘Fog Index’.\nLoop over the DataFrame: The code then loops over each row in final_df. For each row, it does the following:\nSentence length: It calculates the number of sentences in the ‘Text’ column using the nltk.sent_tokenize function and stores the result in the ‘sentence length’ column.\nAverage Sentence Length: It calculates the average sentence length by dividing the total word count (‘word_counts’) by the number of sentences (‘sentence length’) and stores the result in the ‘Average Sentence Length’ column.\nPercentage of Complex words: It calculates the percentage of complex words (words with more than two syllables) by dividing the complex word count (‘complex word count’) by the total word count (‘word_counts’) and stores the result in the ‘Percentage

In [177]:
final_df.head()

,Headline,Text,URL,Transform_Text,word_counts,chara_count,average number of words per sentence,average word length,syllable per word,complex word count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,https://insights.blackcoffer.com/how-is-login-...,when people hear ai they often think about sen...,700,4451,38.888889,6.358571,1421,90,18.0,38.888889,0.128571,15.606984
1,None,None,https://insights.blackcoffer.com/how-does-ai-h...,None,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,AI and its impact on the Fashion Industry,If you were a fan of the 90’s film Clueless ba...,https://insights.blackcoffer.com/ai-and-its-im...,if you were a fan of the 90’s film clueless ba...,1752,11605,44.923077,6.623858,3742,291,39.0,44.923077,0.166096,18.035669
3,None,None,https://insights.blackcoffer.com/how-do-deep-l...,None,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h...",https://insights.blackcoffer.com/how-artificia...,"from the stone age to the modern world, from h...",1208,7545,25.166667,6.245861,2464,167,48.0,25.166667,0.138245,10.121965


In [179]:
# Load the list of positive words
with open('positive-words.txt', 'r') as f:
    positive_words = set(word.strip() for word in f)

def positive_word_score(text):
    # Check if text is not None
    if text is not None:
        # Convert the text to lowercase and split into words
        words = text.lower().split()

        # Count the positive words
        score = sum(1 for word in words if word in positive_words)
    else:
        score = 0

    return score

# Apply the function to the 'Transform_Text' column and store the result in 'Positive_Word_Score'
final_df['Positive_Word_Score'] = final_df['Transform_Text'].apply(positive_word_score)


In [182]:
# Load the list of negative words
with open('negative-words.txt', 'r') as f:
    negative_words = set(word.strip() for word in f)

def negative_word_score(text):
    # Check if text is not None
    if text is not None:
        # Convert the text to lowercase and split into words
        words = text.lower().split()

        # Count the positive words
        score = sum(1 for word in words if word in negative_words)
    else:
        score = 0

    return score

# Apply the function to the 'Transform_Text' column and store the result in 'Positive_Word_Score'
final_df['Nagetive_Word_Score'] = final_df['Transform_Text'].apply(negative_word_score)

In [184]:
def polarity_score(row):
    # Check if text is not None
    if row['Text'] is not None:
        positive_score = row['Positive_Word_Score']
        negative_score = row['Nagetive_Word_Score']

        # Calculate the polarity score
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    else:
        polarity_score = 0  # or any other value you want to assign when text is None

    return polarity_score

# Apply the function to each row in the DataFrame and store the result in 'Polarity_Score'
final_df['Polarity_Score'] = final_df.apply(polarity_score, axis=1)


In [186]:
def subjectivity_score(row):
    # Check if text is not None
    if row['Transform_Text'] is not None:
        positive_score = row['Positive_Word_Score']
        negative_score = row['Nagetive_Word_Score']
        total_words = row['word_counts']

        # Calculate the subjectivity score
        subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
    else:
        subjectivity_score = 0  # or any other value you want to assign when text is None

    return subjectivity_score

# Apply the function to each row in the DataFrame and store the result in 'Subjectivity_Score'
final_df['Subjectivity_Score'] = final_df.apply(subjectivity_score, axis=1)


In [188]:

def personal_pronoun_count(text):
    # Check if text is not None
    if text is not None:
        # Define the personal pronouns
        personal_pronouns = ['I', 'we', 'my', 'ours', 'us']

        # Convert the text to lowercase
        text = text.lower()

        # Count the personal pronouns
        count = sum(1 for word in text.split() if word in personal_pronouns and word != 'us')
        
        # Special case: count 'us' only if it's not part of 'U.S.'
        count += len(re.findall(r'\bus\b', text))
        '''
1>>> re.findall(r'\bus\b', text): This uses the findall function from Python’s re (regular expression) module
to find all occurrences of the word “us” in the text. The r'\bus\b' is a regular expression pattern that matches “us” 
as a whole word (\b denotes a word boundary in regular expressions). This means it won’t match “us” when it’s part of
another word, like “use” or “usual”.

2>>> len(...): This counts the number of matches found by re.findall.

3>>> count += ...: This adds the number of matches to the count variable. '''
    else:
        count = 0

    return count

# Apply the function to the 'Text' column and store the result in 'Personal_Pronoun_Count'
final_df['Personal_Pronoun_Count'] = final_df['Text'].apply(personal_pronoun_count)


In [189]:
final_df.head(5)

,Headline,Text,URL,Transform_Text,word_counts,chara_count,average number of words per sentence,average word length,syllable per word,complex word count,sentence length,Average Sentence Length,Percentage of Complex words,Fog Index,Positive_Word_Score,Nagetive_Word_Score,Polarity_Score,Subjectivity_Score,Personal_Pronoun_Count
0,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...,https://insights.blackcoffer.com/how-is-login-...,when people hear ai they often think about sen...,700,4451,38.888889,6.358571,1421,90,18.0,38.888889,0.128571,15.606984,14,8,0.272727,0.031429,4
1,None,None,https://insights.blackcoffer.com/how-does-ai-h...,None,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0.000000,0.000000,0
2,AI and its impact on the Fashion Industry,If you were a fan of the 90’s film Clueless ba...,https://insights.blackcoffer.com/ai-and-its-im...,if you were a fan of the 90’s film clueless ba...,1752,11605,44.923077,6.623858,3742,291,39.0,44.923077,0.166096,18.035669,75,15,0.666667,0.051370,10
3,None,None,https://insights.blackcoffer.com/how-do-deep-l...,None,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0.000000,0.000000,0
4,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h...",https://insights.blackcoffer.com/how-artificia...,"from the stone age to the modern world, from h...",1208,7545,25.166667,6.245861,2464,167,48.0,25.166667,0.138245,10.121965,46,8,0.703704,0.044702,24


In [191]:
final_df.to_excel('Output.xlsx')